In [1]:
pip install xgboost lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [8]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/Evan-Data/MNCAA_train_538.csv'
file_key_2 = 'March-Mania-2023/Evan-Data/MNCAA_test_538.csv'
file_key_3 = 'March-Mania-2023/Evan-Data/WNCAA_train_538.csv'
file_key_4 = 'March-Mania-2023/Evan-Data/WNCAA_test_538.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
# man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
# woman_test = pd.read_csv(file_content_stream_4)

man_train['target'] = np.where(man_train['ResultDiff'] > 0, 1, 0)
man_train = man_train[man_train['Season'] >= 2016].reset_index(drop = True)

woman_train['target'] = np.where(woman_train['ResultDiff'] > 0, 1, 0)
woman_train = woman_train[woman_train['Season'] >= 2016].reset_index(drop = True)

man_test = pd.read_csv('man_test_tour_phase_1.csv')
woman_test = pd.read_csv('woman_test_tour_phase_1.csv')

# Man

In [12]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2', 
             'ResultDiff',
             'X1_team_rating',
             'X1_rd1_win',
             'X1_rd2_win',
             'X1_rd3_win',
             'X1_rd4_win',
             'X1_rd5_win',
             'X1_rd6_win',
             'X1_rd7_win',
             'X2_team_rating',
             'X2_rd1_win',
             'X2_rd2_win',
             'X2_rd3_win',
             'X2_rd4_win',
             'X2_rd5_win',
             'X2_rd6_win',
             'X2_rd7_win']

X = man_train[to_select]
Y = man_train['target']

man_test_tour_ID = man_test['ID']
man_test_tour = man_test[to_select]

In [15]:
man_test_tour

,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,ResultDiff,X1_team_rating,X1_rd1_win,X1_rd2_win,X1_rd3_win,X1_rd4_win,X1_rd5_win,X1_rd6_win,X1_rd7_win,X2_team_rating,X2_rd1_win,X2_rd2_win,X2_rd3_win,X2_rd4_win,X2_rd5_win,X2_rd6_win,X2_rd7_win
0,0.80,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.528760,75.384559,1.884614,0.275133,0.179410,0.740689,0.800000,82.676471,84.5,11.176471,59.264706,59.0,49,71,19.176471,3.205882,63.294118,45,0.569708,73.827206,1.845680,0.265131,0.176279,0.729807,1.0,2.0,-1.0,1.313516,1.376789,4,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,88.98,1.000000,0.941026,0.676923,0.351350,0.149377,0.085218,0.036114
1,0.80,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.528760,75.384559,1.884614,0.275133,0.179410,0.740689,0.400000,70.323529,68.5,2.617647,60.470588,62.0,47,75,14.235294,4.676471,58.294118,49,0.481441,71.490441,1.787261,0.220176,0.160698,0.680526,1.0,11.0,-10.0,1.313516,0.950975,12,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,80.36,0.544272,0.183005,0.050300,0.019230,0.007441,0.001812,0.000619
2,0.80,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.528760,75.384559,1.884614,0.275133,0.179410,0.740689,0.250000,74.424242,76.0,7.030303,56.909091,56.0,48,70,13.424242,5.242424,55.757576,44,0.522086,71.236364,1.780909,0.270631,0.176877,0.736434,1.0,8.0,-7.0,1.313516,0.834293,10,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,85.12,1.000000,0.516337,0.179549,0.086244,0.035891,0.013111,0.006306
3,0.80,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.528760,75.384559,1.884614,0.275133,0.179410,0.740689,0.333333,72.718750,72.0,5.656250,58.593750,59.0,47,73,14.187500,5.093750,56.437500,47,0.495432,70.496094,1.762402,0.255005,0.167288,0.687117,1.0,9.0,-8.0,1.313516,0.937026,11,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,84.86,1.000000,0.579162,0.146244,0.082779,0.037729,0.017131,0.007497
4,0.80,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.528760,75.384559,1.884614,0.275133,0.179410,0.740689,0.333333,77.156250,77.0,6.906250,57.593750,56.5,50,72,14.500000,2.375000,56.187500,45,0.530645,69.980469,1.749512,0.279926,0.170526,0.667339,1.0,3.0,-2.0,1.313516,0.979970,5,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,87.12,1.000000,0.889041,0.451510,0.241393,0.097918,0.057396,0.025092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,1.00,72.156250,74.0,5.437500,55.562500,55.5,46,69,13.656250,2.937500,55.156250,47,0.550682,66.652344,1.666309,0.201526,0.132953,0.764322,0.750000,76.303030,76.0,5.272727,57.484848,57.0,50,67,13.151515,3.090909,54.909091,42,0.517366,70.899242,1.772481,0.297621,0.174426,0.720425,15.0,9.0,6.0,0.927077,0.899965,-5,76.24,1.0,0.110532,0.027714,0.007624,0.001653,0.000270,0.000042,84.14,1.000000,0.492525,0.091416,0.054639,0.026163,0.011506,0.003678
2274,1.00,72.156250,74.0,5.437500,55.562500,55.5,46,69,13.656250,2.937500,55.156250,47,0.550682,66.652344,1.666309,0.201526,0.132953,0.764322,0.800000,81.411765,82.0,7.352941,61.088235,61.0,50,71,19.264706,3.176471,61.470588,46,0.558353,73.323529,1.833088,0.224991,0.168546,0.760575,15.0,3.0,12.0,0.927077,1.031893,-8,76.24,1.0,0.110532,0.027714,0.007624,0

In [20]:
#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_2_42_Optuna_Hyperparameters.csv')

xgb_md = XGBClassifier(tree_method = 'hist', 
                       max_depth = xgb_params['max_depth'][0],
                       learning_rate = xgb_params['learning_rate'][0],
                       n_estimators = xgb_params['n_estimators'][0],
                       gamma = xgb_params['gamma'][0],
                       min_child_weight = xgb_params['min_child_weight'][0],
                       colsample_bytree = xgb_params['colsample_bytree'][0],
                       subsample = xgb_params['subsample'][0]).fit(X, Y)

xgb_pred = xgb_md.predict_proba(man_test_tour)[:, 1]

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_2_42_Optuna_Hyperparameters.csv')

lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                        n_estimators = lgb_params['n_estimators'][0],
                        learning_rate = lgb_params['learning_rate'][0],
                        max_depth = lgb_params['max_depth'][0],
                        lambda_l1 = lgb_params['lambda_l1'][0],
                        lambda_l2 = lgb_params['lambda_l2'][0],
                        num_leaves = lgb_params['num_leaves'][0],
                        bagging_fraction = lgb_params['bagging_fraction'][0],
                        feature_fraction = lgb_params['feature_fraction'][0]).fit(X, Y)

lgb_pred = lgb_md.predict_proba(man_test_tour)[:, 1]

##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_1_42_Optuna_Hyperparameters.csv')

hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                         early_stopping = False,
                                         learning_rate = hist_params['learning_rate'][0],
                                         max_iter = hist_params['max_iter'][0],
                                         max_depth = hist_params['max_depth'][0],
                                         max_bins = hist_params['max_bins'][0],
                                         min_samples_leaf = hist_params['min_samples_leaf'][0],
                                         max_leaf_nodes = hist_params['max_leaf_nodes'][0]).fit(X, Y)

hist_pred = hist_md.predict_proba(man_test_tour)[:, 1]

[LightGBM] [Warning] lambda_l1 is set=0.2260228224184179, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2260228224184179
[LightGBM] [Warning] feature_fraction is set=0.749437638037727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.749437638037727
[LightGBM] [Warning] lambda_l2 is set=2.6940257639572143, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6940257639572143
[LightGBM] [Warning] bagging_fraction is set=0.5844471963342652, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5844471963342652


In [18]:
xgb_pred

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5], dtype=float32)

In [19]:
lgb_pred

array([0.99941535, 0.99947264, 0.99947264, ..., 0.99947264, 0.99936685,
       0.9993514 ])

In [21]:
hist_pred

array([0.9665203, 0.9665203, 0.9665203, ..., 0.9665203, 0.9665203,
       0.9665203])